In [32]:
from tensorflow import keras
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.models import load_model

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import re
import csv
import tqdm.notebook as tq
import glob
import os

## Read Dataset

In [4]:
def read_file( file_name ):
  print('reading' , file_name )
  f =  open( file_name ,'r')
  lines = f.readlines()
  x = []
  y = []
  for line in tq.tqdm( lines ):
    line = line.rstrip()
    line = line.split(',')
    y.append( int( line[0] ) )
    x.append(  ' '.join( line[1:] )  )

  #convert y to one hot vector

  y = np.array(y,dtype = int)
  b = np.zeros( ( y.size, y.max()+1)  )
  b[np.arange(y.size),y] = 1
  y = b
  return x,y

## Change Paths Accordingly

In [21]:
current_dir = '/content/drive/My Drive/LSTM-deneme/'

train_path = '/content/drive/My Drive/LSTM-deneme/train.csv'
test_path  = '/content/drive/My Drive/LSTM-deneme/test.csv'

In [6]:
train_x, train_y = read_file( train_path )
test_x, test_y  = read_file( test_path  )

n_class = train_y.shape[1]

reading /content/drive/My Drive/LSTM-deneme/train.csv



reading /content/drive/My Drive/LSTM-deneme/test.csv


## Preprocessing

In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_num_words = 500

# create the tokenizer
t = Tokenizer( num_words = max_num_words )

# fit the tokenizer on the documents
t.fit_on_texts( train_x )

train_x = t.texts_to_matrix( train_x , mode='count')

print( "shape of x: " , train_x.shape )
#print("x0 : " , x[0])
print( train_y.shape )

shape of x:  (574075, 500)
(574075, 10)


## Define Model

In [35]:
embed_dim = 128
lstm_out = 128
batch_size = 512
lr = 0.001

model = Sequential()
model.add( Embedding( train_x.shape[1], embed_dim, input_length = train_x.shape[1] ) )
model.add( LSTM( lstm_out ) )
model.add( Dense( n_class ,activation='softmax') )

opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile( loss = 'categorical_crossentropy', optimizer=opt ,metrics = ['accuracy'] )
print( model.summary() )

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 128)          64000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 196,874
Trainable params: 196,874
Non-trainable params: 0
_________________________________________________________________
None


## Define Checkpoint

In [36]:
experiments = glob.glob( current_dir + 'experiment*' )

experiment_name = 'experiment-' + str( len(experiments) + 1 )

if not os.path.exists( current_dir + experiment_name  ):
    os.makedirs( current_dir + experiment_name )

with open( current_dir + experiment_name + '/model.txt','w') as fh:
    model.summary(print_fn=lambda x: fh.write(x + '\n'))

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( filepath = experiment_name + '/model.{epoch:02d}-{accuracy:.3f}' )

## Train Model !

In [ ]:
model.fit( train_x, train_y, epochs=200, batch_size = batch_size , verbose=1, callbacks = [model_checkpoint_callback] )

Epoch 1/200
1122/1122 [==============================] - 142s 127ms/step - loss: 1.6986 - accuracy: 0.3711
Epoch 2/200
1122/1122 [==============================] - 142s 127ms/step - loss: 1.6917 - accuracy: 0.3714
Epoch 3/200
1122/1122 [==============================] - 143s 127ms/step - loss: 1.6737 - accuracy: 0.3714
Epoch 4/200
1122/1122 [==============================] - 143s 127ms/step - loss: 1.6401 - accuracy: 0.3747
Epoch 5/200
1122/1122 [==============================] - 143s 127ms/step - loss: 1.6889 - accuracy: 0.3721
Epoch 6/200
1122/1122 [==============================] - 143s 127ms/step - loss: 1.6829 - accuracy: 0.3728
Epoch 7/200
1122/1122 [==============================] - 142s 127ms/step - loss: 1.6664 - accuracy: 0.3749
Epoch 8/200
1122/1122 [==============================] - ETA: 0s - loss: 1.6416 - accuracy: 0.3794